# 인터프리터와 컴파일러

지금까지 알아본 프로그래밍 방식으로 정수의 사칙연산에서 나눗셈을 제외한 3칙 연산만 지원하는
간단한 수식에 대한 인터프리터와 컴파일러를 만들어 볼 수 있다.

실제 인터프리터나 컴파일러는 텍스트 문자열로 된 소스코드를 트리의 형태로 변환하는 문법분석(parsing)이 기본적으로 필요하지만 그것은 고학년에 프로그래밍 언어나 컴파일러 등을 다루는 전공과목 수업을 들을 경우 자세히 배울 수 있으므로 여기서는 이미 문법분석이 다 되어서 트리의 형태로 수식을 다룰 수 있다고 가정하자. 수식 타입을 이진트리와 마찬가지 방식으로 아래와 같은 데이타 타입으로 정의할 수 있다. 수식을 보통 영어로 expression이라고 하므로 그 앞글자를 따서 타입의 이름을 `Exp`로 붙였다.

In [1]:
module Syntax where

data Exp = Num Int        -- "4"      (Num 4)
         | Plus Exp Exp   -- "4 + 3"  (Plus (Num 4) (Num 3))
         | Minus Exp Exp  -- "4 - 3"  (Minus (Num 4) (Num 3))
         | Times Exp Exp  -- "4 * 3"  (Times (Num 4) (Num 3))
         deriving Show

In [2]:
-- "3 + (5 - 2)"를 아래와 같이 나타낸다
Plus (Num 3) (Minus (Num 5) (Num 2))

Plus (Num 3) (Minus (Num 5) (Num 2))

In [3]:
:type Plus (Num 3) (Minus (Num 5) (Num 2))

Plus (Num 3) (Minus (Num 5) (Num 2)) :: Exp

# Lexer와 Parser 생성기

Haskell에서 alex와 happy가 바로 C에서 lex와 yacc에 해당하는 프로그램 생성기다.
숙련된 개발자가 터미널에서 작업한다면 Makefile 같은 걸 만들어 쓰겠지만
우리는 그냥 노트북 인터페이스에서 `ghci` 특수 명령 또는 `System.Process` 모듈을 통해 외부 명령어를 프로세스를 호출해서 쓰기로 하자.

우선 :으로 시작하는 `ghci` 명령어 중에 느낌표(!)는 외부 명령어를 실행할 수 있다. Docker에 설치된 리눅스의 유틸리티인 `which`를 사용해 다른 ghc 컴파일러나 다른 유틸리티가 어느 경로에 설치되어 있으며 알아볼 수 있으며 개략적인 사용법을 알아보기 위해 각각을 `--help` 옵션으로 호출해 볼 수도 있다.

In [4]:
:! which sed ghc alex happy

/bin/sed
/root/.stack/programs/x86_64-linux/ghc-7.10.2/bin/ghc
/root/.stack/snapshots/x86_64-linux/nightly-2015-08-15/7.10.2/bin/alex
/root/.stack/snapshots/x86_64-linux/nightly-2015-08-15/7.10.2/bin/happy

In [5]:
import System.Process

callProcess "which" ["sed","ghc","alex","happy"]

/bin/sed
/root/.stack/programs/x86_64-linux/ghc-7.10.2/bin/ghc
/root/.stack/snapshots/x86_64-linux/nightly-2015-08-15/7.10.2/bin/alex
/root/.stack/snapshots/x86_64-linux/nightly-2015-08-15/7.10.2/bin/happy

In [6]:
:! alex --help

Usage: alex [OPTION...] file

  -o FILE   --outfile=FILE  write the output to FILE (default: file.hs)
  -i[FILE]  --info[=FILE]   put detailed state-machine info in FILE (or file.info)
  -t DIR    --template=DIR  look in DIR for template files
  -g        --ghc           use GHC extensions
  -l        --latin1        generated lexer will use the Latin-1 encoding instead of UTF-8
  -d        --debug         produce a debugging scanner
  -?        --help          display this help and exit
  -V, -v    --version       output version information and exit

In [7]:
callProcess "happy" ["--help"]

Usage: happy [OPTION...] file

  -o FILE   --outfile=FILE     write the output to FILE (default: file.hs)
  -i[FILE]  --info[=FILE]      put detailed grammar info in FILE
  -t DIR    --template=DIR     look in DIR for template files
  -m NAME   --magic-name=NAME  use NAME as the symbol prefix instead of "happy"
  -s        --strict           evaluate semantic values strictly (experimental)
  -g        --ghc              use GHC extensions
  -c        --coerce           use type coercions (only available with -g)
  -a        --array            generate an array-based parser
  -d        --debug            produce a debugging parser (only with -a)
  -l        --glr              Generate a GLR parser for ambiguous grammars
  -k        --decode           Generate simple decoding code for GLR result
  -f        --filter           Filter the GLR parse forest with respect to semantic usage
  -?        --help             display this help and exit
  -V, -v    --version          output version i

In [8]:
module Token where

data Token
  = Int Int
  | Bop Char  -- +. -, *, /
  | LParen    -- (
  | RParen    -- )
  deriving (Eq,Show)

In [9]:
:! ls

InterpreterAndCompiler.ipynb
Lexer.hi
Lexer.hs
Lexer.hspp
Lexer.o
Lexer.x
Parser.hi
Parser.hs
Parser.hspp
Parser.o
Parser.y
Syntax.hi
Syntax.hs
Syntax.o
Token.hi
Token.hs
Token.o

In [10]:
callProcess "alex" ["Lexer.x"]
callProcess "ghc" ["-E", "Lexer.hs"]
writeFile "Lexer.hs" =<< readProcess "sed" ["/^#/d;/^{-#.*#-}$/d;/^$/d","Lexer.hspp"] ""
:! ghc -Wnot -c Lexer.hs
{- 여기까지 잘 진행되었으면 Lexer.hs 내용을 아래 칸에 드래그 앤 드롭으로 떨군다.
   Lexer.x 를 다시 수정해서 생성할 때는 아래 칸의 내용을 지우고 다시 떨구며 됨. -}

In [11]:
module Lexer (lexer) where
import Token
import Data.Array
import Data.Char (ord)
import Data.Array.Base (unsafeAt)
-- -----------------------------------------------------------------------------
-- Alex wrapper code.
--
-- This code is in the PUBLIC DOMAIN; you may copy it freely and use
-- it for any purpose whatsoever.
import Control.Applicative (Applicative (..))
import Data.Word (Word8)
import qualified Data.Bits
-- | Encode a Haskell String to a list of Word8 values, in UTF8 format.
utf8Encode :: Char -> [Word8]
utf8Encode = map fromIntegral . go . ord
 where
  go oc
   | oc <= 0x7f       = [oc]
   | oc <= 0x7ff      = [ 0xc0 + (oc `Data.Bits.shiftR` 6)
                        , 0x80 + oc Data.Bits..&. 0x3f
                        ]
   | oc <= 0xffff     = [ 0xe0 + (oc `Data.Bits.shiftR` 12)
                        , 0x80 + ((oc `Data.Bits.shiftR` 6) Data.Bits..&. 0x3f)
                        , 0x80 + oc Data.Bits..&. 0x3f
                        ]
   | otherwise        = [ 0xf0 + (oc `Data.Bits.shiftR` 18)
                        , 0x80 + ((oc `Data.Bits.shiftR` 12) Data.Bits..&. 0x3f)
                        , 0x80 + ((oc `Data.Bits.shiftR` 6) Data.Bits..&. 0x3f)
                        , 0x80 + oc Data.Bits..&. 0x3f
                        ]
type Byte = Word8
-- -----------------------------------------------------------------------------
-- The input type
-- -----------------------------------------------------------------------------
-- Token positions
-- `Posn' records the location of a token in the input text.  It has three
-- fields: the address (number of chacaters preceding the token), line number
-- and column of a token within the file. `start_pos' gives the position of the
-- start of the file and `eof_pos' a standard encoding for the end of file.
-- `move_pos' calculates the new position after traversing a given character,
-- assuming the usual eight character tab stops.
-- -----------------------------------------------------------------------------
-- Default monad
-- -----------------------------------------------------------------------------
-- Monad (with ByteString input)
-- -----------------------------------------------------------------------------
-- Basic wrapper
type AlexInput = (Char,[Byte],String)
alexInputPrevChar :: AlexInput -> Char
alexInputPrevChar (c,_,_) = c
-- alexScanTokens :: String -> [token]
alexScanTokens str = go ('\n',[],str)
  where go inp@(_,_bs,s) =
          case alexScan inp 0 of
                AlexEOF -> []
                AlexError _ -> error "lexical error"
                AlexSkip  inp' len     -> go inp'
                AlexToken inp' len act -> act (take len s) : go inp'
alexGetByte :: AlexInput -> Maybe (Byte,AlexInput)
alexGetByte (c,(b:bs),s) = Just (b,(c,bs,s))
alexGetByte (c,[],[])    = Nothing
alexGetByte (_,[],(c:s)) = case utf8Encode c of
                             (b:bs) -> Just (b, (c, bs, s))
                             [] -> Nothing
-- -----------------------------------------------------------------------------
-- Basic wrapper, ByteString version
-- -----------------------------------------------------------------------------
-- Posn wrapper
-- Adds text positions to the basic model.
-- -----------------------------------------------------------------------------
-- Posn wrapper, ByteString version
-- -----------------------------------------------------------------------------
-- GScan wrapper
-- For compatibility with previous versions of Alex, and because we can.
alex_base :: Array Int Int
alex_base = listArray (0,5) [-8,-3,0,0,0,-37]
alex_table :: Array Int Int
alex_table = listArray (0,252) [0,1,1,1,1,1,1,1,1,1,1,5,5,5,5,5,5,5,5,5,5,0,0,0,1,0,0,0,0,1,0,0,2,3,4,4,0,4,0,0,5,5,5,5,5,5,5,5,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
alex_check :: Array Int Int
alex_check = listArray (0,252) [-1,9,10,11,12,13,9,10,11,12,13,48,49,50,51,52,53,54,55,56,57,-1,-1,-1,32,-1,-1,-1,-1,32,-1,-1,40,41,42,43,-1,45,-1,-1,48,49,50,51,52,53,54,55,56,57,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
alex_deflt :: Array Int Int
alex_deflt = listArray (0,5) [-1,-1,-1,-1,-1,-1]
alex_accept = listArray (0::Int,5) [AlexAccNone,AlexAccSkip,AlexAcc (alex_action_1),AlexAcc (alex_action_2),AlexAcc (alex_action_3),AlexAcc (alex_action_4)]
lexer = alexScanTokens
alex_action_1 =  \_     -> LParen 
alex_action_2 =  \_     -> RParen 
alex_action_3 =  \(c:_) -> Bop c 
alex_action_4 =  \s     -> Int (read s) 
-- -----------------------------------------------------------------------------
-- ALEX TEMPLATE
--
-- This code is in the PUBLIC DOMAIN; you may copy it freely and use
-- it for any purpose whatsoever.
-- -----------------------------------------------------------------------------
-- INTERNALS and main scanner engine
alexIndexInt16OffAddr arr off = arr ! off
alexIndexInt32OffAddr arr off = arr ! off
quickIndex arr i = arr ! i
-- -----------------------------------------------------------------------------
-- Main lexing routines
data AlexReturn a
  = AlexEOF
  | AlexError  !AlexInput
  | AlexSkip   !AlexInput !Int
  | AlexToken  !AlexInput !Int a
-- alexScan :: AlexInput -> StartCode -> AlexReturn a
alexScan input (sc)
  = alexScanUser undefined input (sc)
alexScanUser user input (sc)
  = case alex_scan_tkn user input (0) input sc AlexNone of
	(AlexNone, input') ->
		case alexGetByte input of
			Nothing -> 
				   AlexEOF
			Just _ ->
				   AlexError input'
	(AlexLastSkip input'' len, _) ->
		AlexSkip input'' len
	(AlexLastAcc k input''' len, _) ->
		AlexToken input''' len k
-- Push the input through the DFA, remembering the most recent accepting
-- state it encountered.
alex_scan_tkn user orig_input len input s last_acc =
  input `seq` -- strict in the input
  let 
	new_acc = (check_accs (alex_accept `quickIndex` (s)))
  in
  new_acc `seq`
  case alexGetByte input of
     Nothing -> (new_acc, input)
     Just (c, new_input) -> 
      case fromIntegral c of { (ord_c) ->
        let
                base   = alexIndexInt32OffAddr alex_base s
                offset = (base + ord_c)
                check  = alexIndexInt16OffAddr alex_check offset
		
                new_s = if (offset >= (0)) && (check == ord_c)
			  then alexIndexInt16OffAddr alex_table offset
			  else alexIndexInt16OffAddr alex_deflt s
	in
        case new_s of
	    (-1) -> (new_acc, input)
		-- on an error, we want to keep the input *before* the
		-- character that failed, not after.
    	    _ -> alex_scan_tkn user orig_input (if c < 0x80 || c >= 0xC0 then (len + (1)) else len)
                                                -- note that the length is increased ONLY if this is the 1st byte in a char encoding)
			new_input new_s new_acc
      }
  where
	check_accs (AlexAccNone) = last_acc
	check_accs (AlexAcc a  ) = AlexLastAcc a input (len)
	check_accs (AlexAccSkip) = AlexLastSkip  input (len)
	check_accs (AlexAccPred a predx rest)
	   | predx user orig_input (len) input
	   = AlexLastAcc a input (len)
	   | otherwise
	   = check_accs rest
	check_accs (AlexAccSkipPred predx rest)
	   | predx user orig_input (len) input
	   = AlexLastSkip input (len)
	   | otherwise
	   = check_accs rest
data AlexLastAcc a
  = AlexNone
  | AlexLastAcc a !AlexInput !Int
  | AlexLastSkip  !AlexInput !Int
instance Functor AlexLastAcc where
    fmap f AlexNone = AlexNone
    fmap f (AlexLastAcc x y z) = AlexLastAcc (f x) y z
    fmap f (AlexLastSkip x y) = AlexLastSkip x y
data AlexAcc a user
  = AlexAccNone
  | AlexAcc a
  | AlexAccSkip
  | AlexAccPred a   (AlexAccPred user) (AlexAcc a user)
  | AlexAccSkipPred (AlexAccPred user) (AlexAcc a user)
type AlexAccPred user = user -> AlexInput -> Int -> AlexInput -> Bool
-- -----------------------------------------------------------------------------
-- Predicates on a rule
alexAndPred p1 p2 user in1 len in2
  = p1 user in1 len in2 && p2 user in1 len in2
--alexPrevCharIsPred :: Char -> AlexAccPred _ 
alexPrevCharIs c _ input _ _ = c == alexInputPrevChar input
alexPrevCharMatches f _ input _ _ = f (alexInputPrevChar input)
--alexPrevCharIsOneOfPred :: Array Char Bool -> AlexAccPred _ 
alexPrevCharIsOneOf arr _ input _ _ = arr ! alexInputPrevChar input
--alexRightContext :: Int -> AlexAccPred _
alexRightContext (sc) user _ _ input = 
     case alex_scan_tkn user input (0) input sc AlexNone of
	  (AlexNone, _) -> False
	  _ -> True
	-- TODO: there's no need to find the longest
	-- match when checking the right context, just
	-- the first match will do.
-- used by wrappers
iUnbox (i) = i





Line 61: Redundant bracket
Found:
(c, (b : bs), s)
Why not:
(c, b : bs, s)Line 63: Redundant bracket
Found:
(_, [], (c : s))
Why not:
(_, [], c : s)Line 77: Use camelCase
Found:
alex_base = ...
Why not:
alexBase = ...Line 79: Use camelCase
Found:
alex_table = ...
Why not:
alexTable = ...Line 81: Use camelCase
Found:
alex_check = ...
Why not:
alexCheck = ...Line 83: Use camelCase
Found:
alex_deflt = ...
Why not:
alexDeflt = ...Line 84: Use camelCase
Found:
alex_accept = ...
Why not:
alexAccept = ...Line 84: Redundant bracket
Found:
(alex_action_1)
Why not:
alex_action_1Line 84: Redundant bracket
Found:
(alex_action_2)
Why not:
alex_action_2Line 84: Redundant bracket
Found:
(alex_action_3)
Why not:
alex_action_3Line 84: Redundant bracket
Found:
(alex_action_4)
Why not:
alex_action_4Line 86: Redundant lambda
Found:
alex_action_1 = \ _ -> LParen
Why not:
alex_action_1 _ = LParenLine 86: Use const
Found:
\ _ -> LParen
Why not:
const LParenLine 87: Redundant lambda
Found:
alex_action_2 = \ _ -> RParen
Why not:
alex_action_2 _ = RParenLine 87: Use const
Found:
\ _ -> RParen
Why not:
const RParenLine 88: Redundant lambda
Found:
alex_action_3 = \ (c : _) -> Bop c
Why not:
alex_action_3 (c : _) = Bop cLine 89: Redundant lambda
Found:
alex_action_4 = \ s -> Int (read s)
Why not:
alex_action_4 s = Int (read s)Line 89: Avoid lambda
Found:
\ s -> Int (read s)
Why not:
Int . readLine 109: Redundant bracket
Found:
(sc)
Why not:
scLine 111: Redundant bracket
Found:
(0)
Why not:
0Line 124: Use camelCase
Found:
alex_scan_tkn user orig_input len input s last_acc = ...
Why not:
alexScanTkn user orig_input len input s last_acc = ...Line 126: Redundant bracket
Found:
let new_acc = (check_accs (alex_accept `quickIndex` (s))) in
  new_acc `seq`
    case alexGetByte input of
        Nothing -> (new_acc, input)
        Just (c, new_input) -> case fromIntegral c of
                                   (ord_c) -> let base = alexIndexInt32OffAddr alex_base s
                                                  offset = (base + ord_c)
                                                  check = alexIndexInt16OffAddr alex_check offset
                                                  new_s
                                                    = if (offset >= (0)) && (check == ord_c) then
                                                        alexIndexInt16OffAddr alex_table offset else
                                                        alexIndexInt16OffAddr alex_deflt s
                                                in
                                                case new_s of
                                                    (-1) -> (new_acc, input)
                                                    _ -> alex_scan_tkn user orig_input
                                                           (if c < 128 || c >= 192 then (len + (1))
                                                              else len)
                                                           new_input
                                                           new_s
                                                           new_acc
Why not:
let new_acc = check_accs (alex_accept `quickIndex` (s)) in
  new_acc `seq`
    case alexGetByte input of
        Nothing -> (new_acc, input)
        Just (c, new_input) -> case fromIntegral c of
                                   (ord_c) -> let base = alexIndexInt32OffAddr alex_base s
                                                  offset = (base + ord_c)
                                                  check = alexIndexInt16OffAddr alex_check offset
                                                  new_s
                                                    = if (offset >= (0)) && (check == ord_c) then
                                                        alexIndexInt16OffAddr alex_table offset else
                                                        alexIndexInt16OffAddr alex_deflt s
                                                in
       

In [12]:
:t lexer

lexer :: String -> [Token]

In [13]:
lexer " 4 * 3 + 5 - 2 "

[Int 4,Bop '*',Int 3,Bop '+',Int 5,Bop '-',Int 2]

이번엔 happy로 parser를 생성할 차례다.

In [14]:
callProcess "happy" ["Parser.y"]
callProcess "ghc" ["-E", "Parser.hs"]
writeFile "Parser.hs" =<< readProcess "sed" ["/^#/d;/^{-#.*#-}$/d;/^$/d","Parser.hspp"] ""
:! ghc -Wnot -c Parser.hs
{- 여기까지 잘 진행되었으면 Parser.hs 내용을 아래 칸에 드래그 앤 드롭으로 떨군다.
   Lexer.x 를 다시 수정해서 생성할 때는 아래 칸의 내용을 지우고 다시 떨구며 됨. -}

In [15]:

module Parser where
import Token
import Lexer
import Syntax
import Control.Applicative(Applicative(..))
import Control.Monad (ap)
-- parser produced by Happy Version 1.19.5
data HappyAbsSyn t4
	= HappyTerminal (Token)
	| HappyErrorToken Int
	| HappyAbsSyn4 t4
action_0 (8) = happyShift action_4
action_0 (10) = happyShift action_2
action_0 (4) = happyGoto action_3
action_0 _ = happyFail
action_1 (10) = happyShift action_2
action_1 _ = happyFail
action_2 _ = happyReduce_1
action_3 (5) = happyShift action_6
action_3 (6) = happyShift action_7
action_3 (7) = happyShift action_8
action_3 (11) = happyAccept
action_3 _ = happyFail
action_4 (8) = happyShift action_4
action_4 (10) = happyShift action_2
action_4 (4) = happyGoto action_5
action_4 _ = happyFail
action_5 (5) = happyShift action_6
action_5 (6) = happyShift action_7
action_5 (7) = happyShift action_8
action_5 (9) = happyShift action_12
action_5 _ = happyFail
action_6 (8) = happyShift action_4
action_6 (10) = happyShift action_2
action_6 (4) = happyGoto action_11
action_6 _ = happyFail
action_7 (8) = happyShift action_4
action_7 (10) = happyShift action_2
action_7 (4) = happyGoto action_10
action_7 _ = happyFail
action_8 (8) = happyShift action_4
action_8 (10) = happyShift action_2
action_8 (4) = happyGoto action_9
action_8 _ = happyFail
action_9 _ = happyReduce_4
action_10 (7) = happyShift action_8
action_10 _ = happyReduce_3
action_11 (7) = happyShift action_8
action_11 _ = happyReduce_2
action_12 _ = happyReduce_5
happyReduce_1 = happySpecReduce_1  4 happyReduction_1
happyReduction_1 (HappyTerminal (Int happy_var_1))
	 =  HappyAbsSyn4
		 (Num happy_var_1
	)
happyReduction_1 _  = notHappyAtAll 
happyReduce_2 = happySpecReduce_3  4 happyReduction_2
happyReduction_2 (HappyAbsSyn4  happy_var_3)
	_
	(HappyAbsSyn4  happy_var_1)
	 =  HappyAbsSyn4
		 (Plus happy_var_1 happy_var_3
	)
happyReduction_2 _ _ _  = notHappyAtAll 
happyReduce_3 = happySpecReduce_3  4 happyReduction_3
happyReduction_3 (HappyAbsSyn4  happy_var_3)
	_
	(HappyAbsSyn4  happy_var_1)
	 =  HappyAbsSyn4
		 (Minus happy_var_1 happy_var_3
	)
happyReduction_3 _ _ _  = notHappyAtAll 
happyReduce_4 = happySpecReduce_3  4 happyReduction_4
happyReduction_4 (HappyAbsSyn4  happy_var_3)
	_
	(HappyAbsSyn4  happy_var_1)
	 =  HappyAbsSyn4
		 (Times happy_var_1 happy_var_3
	)
happyReduction_4 _ _ _  = notHappyAtAll 
happyReduce_5 = happySpecReduce_3  4 happyReduction_5
happyReduction_5 _
	(HappyAbsSyn4  happy_var_2)
	_
	 =  HappyAbsSyn4
		 (happy_var_2
	)
happyReduction_5 _ _ _  = notHappyAtAll 
happyNewToken action sts stk [] =
	action 11 11 notHappyAtAll (HappyState action) sts stk []
happyNewToken action sts stk (tk:tks) =
	let cont i = action i i tk (HappyState action) sts stk tks in
	case tk of {
	Bop '+' -> cont 5;
	Bop '-' -> cont 6;
	Bop '*' -> cont 7;
	LParen -> cont 8;
	RParen -> cont 9;
	Int happy_dollar_dollar -> cont 10;
	_ -> happyError' (tk:tks)
	}
happyError_ 11 tk tks = happyError' tks
happyError_ _ tk tks = happyError' (tk:tks)
newtype HappyIdentity a = HappyIdentity a
happyIdentity = HappyIdentity
happyRunIdentity (HappyIdentity a) = a
instance Functor HappyIdentity where
    fmap f (HappyIdentity a) = HappyIdentity (f a)
instance Applicative HappyIdentity where
    pure  = return
    (<*>) = ap
instance Monad HappyIdentity where
    return = HappyIdentity
    (HappyIdentity p) >>= q = q p
happyThen :: () => HappyIdentity a -> (a -> HappyIdentity b) -> HappyIdentity b
happyThen = (>>=)
happyReturn :: () => a -> HappyIdentity a
happyReturn = (return)
happyThen1 m k tks = (>>=) m (\a -> k a tks)
happyReturn1 :: () => a -> b -> HappyIdentity a
happyReturn1 = \a tks -> (return) a
happyError' :: () => [(Token)] -> HappyIdentity a
happyError' = HappyIdentity . parseError
parse tks = happyRunIdentity happySomeParser where
  happySomeParser = happyThen (happyParse action_0 tks) (\x -> case x of {HappyAbsSyn4 z -> happyReturn z; _other -> notHappyAtAll })
happySeq = happyDontSeq
parseError :: [Token] -> a
parseError _ = error "Parse error"
-- Id: GenericTemplate.hs,v 1.26 2005/01/14 14:47:22 simonmar Exp 
infixr 9 `HappyStk`
data HappyStk a = HappyStk a (HappyStk a)
-----------------------------------------------------------------------------
-- starting the parse
happyParse start_state = happyNewToken start_state notHappyAtAll notHappyAtAll
-----------------------------------------------------------------------------
-- Accepting the parse
-- If the current token is (1), it means we've just accepted a partial
-- parse (a %partial parser).  We must ignore the saved token on the top of
-- the stack in this case.
happyAccept (1) tk st sts (_ `HappyStk` ans `HappyStk` _) =
        happyReturn1 ans
happyAccept j tk st sts (HappyStk ans _) = 
         (happyReturn1 ans)
-----------------------------------------------------------------------------
-- Arrays only: do the next action
-----------------------------------------------------------------------------
-- HappyState data type (not arrays)
newtype HappyState b c = HappyState
        (Int ->                    -- token number
         Int ->                    -- token number (yes, again)
         b ->                           -- token semantic value
         HappyState b c ->              -- current state
         [HappyState b c] ->            -- state stack
         c)
-----------------------------------------------------------------------------
-- Shifting a token
happyShift new_state (1) tk st sts stk@(x `HappyStk` _) =
     let i = (case x of { HappyErrorToken (i) -> i }) in
--     trace "shifting the error token" $
     new_state i i tk (HappyState (new_state)) ((st):(sts)) (stk)
happyShift new_state i tk st sts stk =
     happyNewToken new_state ((st):(sts)) ((HappyTerminal (tk))`HappyStk`stk)
-- happyReduce is specialised for the common cases.
happySpecReduce_0 i fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happySpecReduce_0 nt fn j tk st@((HappyState (action))) sts stk
     = action nt j tk st ((st):(sts)) (fn `HappyStk` stk)
happySpecReduce_1 i fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happySpecReduce_1 nt fn j tk _ sts@(((st@(HappyState (action))):(_))) (v1`HappyStk`stk')
     = let r = fn v1 in
       happySeq r (action nt j tk st sts (r `HappyStk` stk'))
happySpecReduce_2 i fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happySpecReduce_2 nt fn j tk _ ((_):(sts@(((st@(HappyState (action))):(_))))) (v1`HappyStk`v2`HappyStk`stk')
     = let r = fn v1 v2 in
       happySeq r (action nt j tk st sts (r `HappyStk` stk'))
happySpecReduce_3 i fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happySpecReduce_3 nt fn j tk _ ((_):(((_):(sts@(((st@(HappyState (action))):(_))))))) (v1`HappyStk`v2`HappyStk`v3`HappyStk`stk')
     = let r = fn v1 v2 v3 in
       happySeq r (action nt j tk st sts (r `HappyStk` stk'))
happyReduce k i fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happyReduce k nt fn j tk st sts stk
     = case happyDrop (k - ((1) :: Int)) sts of
         sts1@(((st1@(HappyState (action))):(_))) ->
                let r = fn stk in  -- it doesn't hurt to always seq here...
                happyDoSeq r (action nt j tk st1 sts1 r)
happyMonadReduce k nt fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happyMonadReduce k nt fn j tk st sts stk =
      case happyDrop k ((st):(sts)) of
        sts1@(((st1@(HappyState (action))):(_))) ->
          let drop_stk = happyDropStk k stk in
          happyThen1 (fn stk tk) (\r -> action nt j tk st1 sts1 (r `HappyStk` drop_stk))
happyMonad2Reduce k nt fn (1) tk st sts stk
     = happyFail (1) tk st sts stk
happyMonad2Reduce k nt fn j tk st sts stk =
      case happyDrop k ((st):(sts)) of
        sts1@(((st1@(HappyState (action))):(_))) ->
         let drop_stk = happyDropStk k stk
             new_state = action
          in
          happyThen1 (fn stk tk) (\r -> happyNewToken new_state sts1 (r `HappyStk` drop_stk))
happyDrop (0) l = l
happyDrop n ((_):(t)) = happyDrop (n - ((1) :: Int)) t
happyDropStk (0) l = l
happyDropStk n (x `HappyStk` xs) = happyDropStk (n - ((1)::Int)) xs
-----------------------------------------------------------------------------
-- Moving to a new state after a reduction
happyGoto action j tk st = action j j tk (HappyState action)
-----------------------------------------------------------------------------
-- Error recovery ((1) is the error token)
-- parse error if we are in recovery and we fail again
happyFail (1) tk old_st _ stk@(x `HappyStk` _) =
     let i = (case x of { HappyErrorToken (i) -> i }) in
--      trace "failing" $ 
        happyError_ i tk
{-  We don't need state discarding for our restricted implementation of
    "error".  In fact, it can cause some bogus parses, so I've disabled it
    for now --SDM
-- discard a state
happyFail  (1) tk old_st (((HappyState (action))):(sts)) 
                                                (saved_tok `HappyStk` _ `HappyStk` stk) =
--      trace ("discarding state, depth " ++ show (length stk))  $
        action (1) (1) tk (HappyState (action)) sts ((saved_tok`HappyStk`stk))
-}
-- Enter error recovery: generate an error token,
--                       save the old token and carry on.
happyFail  i tk (HappyState (action)) sts stk =
--      trace "entering error recovery" $
        action (1) (1) tk (HappyState (action)) sts ( (HappyErrorToken (i)) `HappyStk` stk)
-- Internal happy errors:
notHappyAtAll :: a
notHappyAtAll = error "Internal Happy error\n"
-----------------------------------------------------------------------------
-- Hack to get the typechecker to accept our action functions
-----------------------------------------------------------------------------
-- Seq-ing.  If the --strict flag is given, then Happy emits 
--      happySeq = happyDoSeq
-- otherwise it emits
--      happySeq = happyDontSeq
happyDoSeq, happyDontSeq :: a -> b -> b
happyDoSeq   a b = a `seq` b
happyDontSeq a b = b
-----------------------------------------------------------------------------
-- Don't inline any functions from the template.  GHC has a nasty habit
-- of deciding to inline happyGoto everywhere, which increases the size of
-- the generated parser quite a bit.
-- end of Happy Template.





Line 86: Redundant bracket
Found:
(happy_var_2)
Why not:
happy_var_2Line 118: Redundant bracket
Found:
(return)
Why not:
returnLine 119: Avoid lambda
Found:
\ a -> k a tks
Why not:
(`k` tks)Line 121: Redundant lambda
Found:
happyReturn1 = \ a tks -> (return) a
Why not:
happyReturn1 a tks = (return) aLine 121: Redundant bracket
Found:
(return)
Why not:
returnLine 122: Redundant bracket
Found:
(Token)
Why not:
TokenLine 143: Redundant bracket
Found:
(happyReturn1 ans)
Why not:
happyReturn1 ansLine 158: Redundant bracket
Found:
(i)
Why not:
iLine 160: Redundant bracket
Found:
(new_state)
Why not:
new_stateLine 160: Redundant bracket
Found:
(st)
Why not:
stLine 160: Redundant bracket
Found:
(sts)
Why not:
stsLine 160: Redundant bracket
Found:
(stk)
Why not:
stkLine 162: Redundant bracket
Found:
(st)
Why not:
stLine 162: Redundant bracket
Found:
(sts)
Why not:
stsLine 162: Redundant bracket
Found:
(HappyTerminal (tk)) `HappyStk` stk
Why not:
HappyTerminal (tk) `HappyStk` stkLine 162: Redundant bracket
Found:
(tk)
Why not:
tkLine 165: Redundant bracket
Found:
(1)
Why not:
1Line 166: Redundant bracket
Found:
((HappyState (action)))
Why not:
(HappyState (action))Line 166: Redundant bracket
Found:
(action)
Why not:
actionLine 167: Redundant bracket
Found:
(st)
Why not:
stLine 167: Redundant bracket
Found:
(sts)
Why not:
stsLine 169: Redundant bracket
Found:
(1)
Why not:
1Line 170: Redundant bracket
Found:
(((st@(HappyState (action))) : (_)))
Why not:
((st@(HappyState (action))) : (_))Line 170: Redundant bracket
Found:
(action)
Why not:
actionLine 170: Redundant bracket
Found:
(_)
Why not:
_Line 174: Redundant bracket
Found:
(1)
Why not:
1Line 175: Redundant bracket
Found:
(_)
Why not:
_Line 175: Redundant bracket
Found:
(((st@(HappyState (action))) : (_)))
Why not:
((st@(HappyState (action))) : (_))Line 175: Redundant bracket
Found:
(action)
Why not:
actionLine 175: Redundant bracket
Found:
(_)
Why not:
_Line 179: Redundant bracket
Found:
(1)
Why not:
1Line 180: Redundant bracket
Found:
(_)
Why not:
_Line 180: Redundant bracket
Found:
(((_) : (sts@(((st@(HappyState (action))) : (_))))))
Why not:
((_) : (sts@(((st@(HappyState (action))) : (_)))))Line 180: Redundant bracket
Found:
(_)
Why not:
_Line 180: Redundant bracket
Found:
(((st@(HappyState (action))) : (_)))
Why not:
((st@(HappyState (action))) : (_))Line 180: Redundant bracket
Found:
(action)
Why not:
actionLine 180: Redundant bracket
Found:
(_)
Why not:
_Line 184: Redundant bracket
Found:
(1)
Why not:
1Line 186: Redundant bracket
Found:
(1)
Why not:
1Line 187: Redundant bracket
Found:
(((st1@(HappyState (action))) : (_)))
Why not:
((st1@(HappyState (action))) : (_))Line 187: Redundant bracket
Found:
(action)
Why not:
actionLine 187: Redundant bracket
Found:
(_)
Why not:
_Line 191: Redundant bracket
Found:
(1)
Why not:
1Line 193: Redundant bracket
Found:
(st)
Why not:
stLine 193: Redundant bracket
Found:
(sts)
Why not:
stsLine 194: Redundant bracket
Found:
(((st1@(HappyState (action))) : (_)))
Why not:
((st1@(HappyState (action))) : (_))Line 194: Redundant bracket
Found:
(action)
Why not:
actionLine 194: Redundant bracket
Found:
(_)
Why not:
_Line 198: Redundant bracket
Found:
(1)
Why not:
1Line 200: Redundant bracket
Found:
(st)
Why not:
stLine 200: Redundant bracket
Found:
(sts)
Why not:
stsLine 201: Redundant bracket
Found:
(((st1@(HappyState (action))) : (_)))
Why not:
((st1@(HappyState (action))) : (_))Line 201: Redundant bracket
Found:
(action)
Why not:
actionLine 201: Redundant bracket
Found:
(_)
Why not:
_Line 207: Redundant bracket
Found:
(_)
Why not:
_Line 207: Redundant bracket
Found:
(t)
Why not:
tLine 207: Redundant bracket
Found:
(1)
Why not:
1Line 209: Redundant bracket
Found:
(1)
Why not:
1Line 217: Redundant bracket
Found:
(i)
Why not:
iLine 231: Redundant bracket
Found:
(action)
Why not:
actionLine 233: Redundant bracket
Found:
(1)
Why not:
1Line 233: Redundant bracket
Found:
(1)
Why not:
1Line 233: Redundant bracket
Found:
(action)
Why not:
actionLine 233: Redu

In [16]:
:type parse

parse :: [Token] -> Exp

In [17]:
:type parse . lexer

parse . lexer :: String -> Exp

In [18]:
parse $ lexer "5 * 2 + 3 - 4"
parse $ lexer "5 + 2 * 3 - 4"

Minus (Plus (Times (Num 5) (Num 2)) (Num 3)) (Num 4)

Minus (Plus (Num 5) (Times (Num 2) (Num 3))) (Num 4)

In [19]:
parse $ lexer "5 + 4 + 3 + 2 + 1"

Plus (Plus (Plus (Plus (Num 5) (Num 4)) (Num 3)) (Num 2)) (Num 1)

In [20]:
parse $ lexer "5 - 4 - 3 - 2 - 1"

Minus (Minus (Minus (Minus (Num 5) (Num 4)) (Num 3)) (Num 2)) (Num 1)

## 인터프리터
인터프리터를 `eval`이라는 함수로 정의하자. 수식의 값을 계산하는 것을 영어로 evaluation이라고 하므로 그 앞부분을 따서 이름을 붙였다.
인터프리터는 `Exp` 타입의 수식을 받아서 `Int` 타입 정수값을 돌려주는 함수이다. 귀납기초인 `(Num n)`은 더 이상 계산할 필요 없이 `n`을 돌려주면 되고 나머지 연산자로 이루어진 귀납단계의 경우들은 두 부분식 `e1`과 `e2`를 계산할 수 있다는 두 개의 귀납가정 `eval e1`과 `eval e2`를 이용하여 아래와 같이 정의하면 된다.

In [22]:
eval :: Exp -> Int
eval (Num n)       = n
eval (Plus  e1 e2) = eval e1 + eval e2
eval (Minus e1 e2) = eval e1 - eval e2
eval (Times e1 e2) = eval e1 * eval e2

In [52]:
-- 3 + (5 - 2) 를 인터프리터로 계산해 보자

eval (Plus (Num 3) (Minus (Num 5) (Num 2)))

eval $ parse $ lexer " 3 + (5 - 2) "

eval $ parse $ lexer "3+(5-2)"

6

6

6

인터프리터는 생각보다 매우 간단하지 않은가? 트리에 대한 구조적 귀납법으로 작성된 함수일 뿐이다. 물론 이렇게까지 간단한 것은 우리 언어가 번수도 함수도 없고 조건문도 반복문도 없는 정말 단순한 언어이기 때문이다. 변수, 함수, 조건문, 반복문 등등의 더 많은 기능이 있는 프로그래밍 언어의 인터프리터는 당연히 더 복잡해지겠지만 기본 핵심 원리는 우리가 만들어본 `eval`과 마찬가지다.

## 컴파일러
컴파일러는 소스 프로그램(혹은 코드)를 목적 코드로 번역해야 한다. 그렇다면 목적 코드에 해당하는 저수준 언어가 있어야 하는데, 여기서는 우리의 소스 프로그램에 쓰이는 3칙연산을 포함한 최소한의 명령어 셋으로 구성된 간단한 가상의 스택 머신을 정의할 것이다. 그래서 우리가 작성할 컴파일러는 Exp 타입의 수식을 받아 이 스택 머신의 기계어 코드를 돌려주는 함수로 정의될 것이다.

### 가상머신에 필요한 타입 정의
일단 스택 머신에는 스택이 있어야 하는데 우리는 지금 다른 값은 고려할 필요 없이 정수만 스택에 넣으면 되므로 그냥 리스트를 사용해 스택을 표현할 수 있다. 예를 들면 `[1,2,3,4]`가 네 개의 값이 들어있는 스택을 표현하는 리스트면 1이 맨 위의 값이고 4가 맨 밑의 값이다.

그 다음으로는 명령어 타입인 `Inst`를 정의하면 된다. 명령어를 영어로 instruction이라고 하므로 그 앞글자들을 따서 이름을 붙였다. 우선 스택머신이므로 `PUSH`와 `POP`이 기본적으로 제공되며 덧셈, 뺄셈, 곱셈에 해당하는 `ADD`, `SUB`, `MUL`이 제공된다. `ADD`, `SUB`, `MUL` 명령어에는 아무런 추가 정보가 필요없다. 왜냐하면 스택머신에서 연산은 스택의 제일 위에 있는 값을 소모하여 연산을 한 후 그 결과값을 스택의 맨 위에다 올려놓는 방식으로, 연산에 필요한 인자의 위치와 연산의 결과를 저장할 위치가 이미 정해져 있기 때문이다.

In [27]:
-- 스택을 정수 리스트로 표현한다. 왼쪽이 꼭대기, 오른쪽이 바닥이다.
type Stack = [Int]

-- 가상 머신의 명령어(인스트럭션)
data Inst = PUSH Int | POP | ADD | SUB | MUL deriving Show

-- 기계어 코드란 명령어(인스트럭션)가 한줄로 나열된 리스트, 즉 [Inst] 타입이다
type Code = [Inst]

### 가상머신 명령어의 의미를 함수로 정의
위에서 `Inst`라는 타입 정의를 통해 가상머신 명령어의 문법을 정의하였다. 각각의 명령어가 어떤 의미를 가지는지, 즉 가상머신에서 명령어 하나를 실행한다는 것이 구체적으로 무슨 의미인지, 즉 어떤 계산으로 표현할 수 있는지를 각각의 명령어의 의미를 명확하게 정의해야 한다. 이렇게 각가의 명령어의 의미를 정의하고 있는 함수가 바로 아래에 있는 `stepVM`이다. `stepVM` 함수는 주어진 가상머신의 스택 상태에서 명령어(인스트럭션) 하나를 실행하고 나면 그 다음 단계에 스택이 어떻게 되어 있을지를 계산하는 함수이다. 즉 스택과 명령어 하나를 받아 스택을 돌려주는 함수인 것이다.

In [28]:
stepVM :: Stack -> Inst -> Stack
stepVM stack       (PUSH n) = n : stack     -- 스택 맨 위에 주어진 값 n 쌓는다
stepVM (n : stack) POP      = stack         -- 스택 맨 위의 값을 꺼내어 버린다
stepVM (x:y:stack) ADD      = (y+x) : stack -- 맨 위의 값 둘을 스택에서 꺼내 그 덧셈 결과를 맨 위에 쌓는다
stepVM (x:y:stack) SUB      = (y-x) : stack -- 맨 위의 값 둘을 스택에서 꺼내 그 뺄셈 결과를 맨 위에 쌓는다
stepVM (x:y:stack) MUL      = (y*x) : stack -- 맨 위의 값 둘을 스택에서 꺼내 그 곱셈 결과를 맨 위에 쌓는다

위에서 정의한 함수로 각각의 명령어가 어떻게 동작하는지 테스트해 보자. 

In [29]:
stepVM [2,3,4] (PUSH 1)

[1,2,3,4]

In [30]:
stepVM [2,3,4] POP

[3,4]

In [31]:
stepVM [2,3,4] ADD

[5,4]

In [32]:
stepVM [2,3,4] SUB

[1,4]

In [33]:
stepVM [2,3,4] MUL

[6,4]

### 손컴파일 예제 한단계씩 실행해 보기
"3 + (5 - 2)"에 해당하는 `(Plus (Num 3) (Minus (Num 5) (Num 2)))`를 손으로 목적 코드(즉, 스택머신 명령어의 나열)로 번역하면 다음과 같이 될 것이다.
```
PUSH 3
PUSH 5
PUSH 2
SUB
ADD
```
이렇게 손컴파일한 것을 빈 스택 상태에서 시작해서 한 단계씩 실행시켜 보자.

In [34]:
stepVM [] (PUSH 3)

[3]

In [35]:
stepVM [3] (PUSH 5)

[5,3]

In [36]:
stepVM [5,3] (PUSH 2)

[2,5,3]

In [37]:
stepVM [2,5,3] SUB

[3,3]

In [38]:
stepVM [3,3] ADD

[6]

### 가상머신 실행
가상머신은 가상머신용 기계어 코드의 인터프리터다. 가상머신은 현재 스택 상태 시작해서 기계어 코드를 실행한다.
기계어 코드는 명령어(인스트럭션)의 나열, 즉 `[Inst]` 타입의 리스트로 표현할 수 있다.
그러므로 가상머신의 실행이란, 현재 가상머신의 상태인 `Stack`과 실행시킬 가상머신용 기계어 `Code`를 받아 실행 후 최종 상태의 `Stack`을 돌려주는 아래와 같은 함수로 정의할 수 있다. 가상머신의 실행을 정의하는 함수 `runVM`은 기계어 코드 길이(명렁어 개수)에 대한 수학적 귀납법 구조를 따라 정의되어 있으며, 앞서 정의한 하나의 명령어를 실행시키는 함수 `stepVM`을 이용하고 있다.

In [39]:
runVM :: Stack -> Code -> Stack
runVM stack []          = stack
runVM stack (inst:code) = runVM (stepVM stack inst) code

In [40]:
runVM [2,3,4] [ADD,ADD]

[9]

In [41]:
-- 손컴파일 예제 목적 코드를 runVM으로 한꺼번에 실행
runVM [] [PUSH 3, PUSH 5, PUSH 2, SUB, ADD]

[6]

### 컴파일러 정의
다음 시간에는 `Exp` 타입의 소스 프로그램을 받아 자동으로 목적 `Code`를 생성하는 컴파일 함수를 정의해 보겠다. 

우선 컴파일러를 정의하기 전에 컴파일러를 정의하는 데 쓸 `(++)` 연산자에 대한 기억을 되살려 보자. 우리가 직접 정의해 사용하던 `List`에 대한 함수들 중 두 리스트를 이어붙이는 `append`라는 함수를 정의했고 이 함수는 `Nat` 타입의 일진수 자연수의 덧셈과 같은 구조를 갖고 있다는 것이 기억나는가? (안난다면 복습을 ...) 하스켈 표준라이브러리에서 제공하는 리스트에 대해 `append` 함수와 같은 역할을 하는 연산자가 바로 `(++)`이다. 일단 하스켈 표준라이브러리 리스트는 (`:`)가 우리가 직접 만들어 쓰던 `List`의 `Cons`에 해당한다. 즉 `(:)`로 다음과 같이 리스트 맨 앞에 원소를 하나 추가할 수 있다.

In [42]:
1 : [2,3]

[1,2,3]

그리고 `(++)`로 두 리스트를 다음과 같이 이어붙일 수 있다.

In [43]:
[2,3] ++ [4,5]

[2,3,4,5]

리스트의 맨 뒤에 원소를 추가할 때도 `(++)` 연산자를 이용할 수 있다. 맨 끝에다 넣을 원소 하나만 들어 있는 리스트를 뒤에다 이어붙이면 된다.

In [44]:
[2,3] ++ [1]

[2,3,1]

리스트를 이어붙이는 `(++)` 연산자 뭔지 이제 기억이 났을 것이다. 혹시 기억이 안나더라도 뭐하는 연산자인지 감을 잡았을 것이다.

이제 compile 함수를 정의하자. 컴파일러는 `Exp` 타입의 소스 프로그램을 받아 목적 `Code`를 돌려주는 함수이다. 여기서 `Code`는 명령어가 일렬로 죽 나열된 리스트, 즉 `[Inst]` 타입을 나타낸다고 앞서 선언했었다. compile 함수의 정의는 다음과 같다.

In [45]:
compile :: Exp -> Code       -- Code = [Inst]
compile (Num n)       = [PUSH n]
compile (Plus  e1 e2) = compile e1 ++ compile e2 ++ [ADD]
compile (Minus e1 e2) = compile e1 ++ compile e2 ++ [SUB]
compile (Times e1 e2) = compile e1 ++ compile e2 ++ [MUL]

In [46]:
compile (Num 3)

[PUSH 3]

In [47]:
compile (Plus (Num 3) (Num 3))

[PUSH 3,PUSH 3,ADD]

In [48]:
runVM [] [PUSH 3,PUSH 3,ADD]

[6]

In [49]:
compile (Plus (Num 3) (Minus (Num 5) (Num 2)))

[PUSH 3,PUSH 5,PUSH 2,SUB,ADD]

In [50]:
eval (Plus (Num 3) (Minus (Num 5) (Num 2)))

6

In [51]:
runVM [] [PUSH 3,PUSH 5,PUSH 2,SUB,ADD]

[6]

In [57]:
eval $ parse $ lexer "3 + (5 - 2)"

compile $ parse $ lexer "3 + (5 -2)"

runVM [] $ compile $ parse $ lexer "3 + (5 -2)"

6

[PUSH 3,PUSH 5,PUSH 2,SUB,ADD]

[6]